In [1]:
import numpy as np
import matplotlib.pyplot as plt

import math
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import regularizers

from tqdm import tqdm
import sys
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


## Basic Models

Create basic model functions
- __Target model__: A small and simple CNN
- __N Shadow Models__: Same architecture as the target model
- __Attack Models__ : One for every class

In [2]:
def f_target(X_train, y_train, X_test=None, y_test=None, epochs=100):
  """
  Returns a trained target model, if test data are specified we will evaluate the model and print its accuracy
  """
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='tanh', input_shape=(32, 32, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(32, (3, 3), activation='tanh'))
  model.add(layers.MaxPooling2D((2, 2)))

  model.add(layers.Flatten())
  model.add(layers.Dense(32, activation='tanh'))

  model.add(layers.Dense(10)   )
  
  optimizer = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  if X_test is None or y_test is None:
    history = model.fit(X_train, y_train, epochs=epochs, 
                    validation_split=0.2)
  else:
    history = model.fit(X_train, y_train, epochs=epochs, 
                    validation_data=(X_test, y_test))
  return model

In [3]:
with tf.device('/gpu:0'):
  (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
  # use the rest as testing - 'out' records
  test_labels = np.concatenate((train_labels[2500:], test_labels))
  test_images = np.concatenate((train_images[2500:], test_images))

  train_images = train_images[:2500] # as the paper attack train wiht only 200 records
  train_labels = train_labels[:2500]


In [4]:
with tf.device('/gpu:0'):
  target_model = f_target(train_images, train_labels, epochs=50) 

Epoch 1/50
63/63 [==============================] - 2s 11ms/step - loss: 2.0759 - accuracy: 0.2450 - val_loss: 1.8743 - val_accuracy: 0.2880
Epoch 2/50
63/63 [==============================] - 0s 7ms/step - loss: 1.7590 - accuracy: 0.3750 - val_loss: 1.7601 - val_accuracy: 0.3720
Epoch 3/50
63/63 [==============================] - 0s 7ms/step - loss: 1.6085 - accuracy: 0.4300 - val_loss: 1.6831 - val_accuracy: 0.3900
Epoch 4/50
63/63 [==============================] - 0s 7ms/step - loss: 1.4991 - accuracy: 0.4605 - val_loss: 1.6659 - val_accuracy: 0.4140
Epoch 5/50
63/63 [==============================] - 0s 7ms/step - loss: 1.3728 - accuracy: 0.5210 - val_loss: 1.6770 - val_accuracy: 0.3920
Epoch 6/50
63/63 [==============================] - 0s 7ms/step - loss: 1.2578 - accuracy: 0.5750 - val_loss: 1.6345 - val_accuracy: 0.4300
Epoch 7/50
63/63 [==============================] - 0s 7ms/step - loss: 1.1513 - accuracy: 0.6075 - val_loss: 1.6009 - val_accuracy: 0.4220
Epoch 8/50
63/63 [=

In [40]:
def f_shadow(X_train, y_train, X_test=None, y_test=None, epochs=100):
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.5))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.5))

  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))

  model.add(layers.Dense(10)   )
  
  optimizer = keras.optimizers.Adam()
  model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  if X_test is None or y_test is None:
    history = model.fit(X_train, y_train, epochs=epochs, 
                    validation_split=0.2)
  else:
    history = model.fit(X_train, y_train, epochs=epochs, 
                    validation_data=(X_test, y_test))
  return model

In [41]:

def divide_dataset(n_shadows, shadow_dataset_size, X, y):
  D_shadows = []
  rng = np.random.default_rng()
  for i in range(n_shadows):
    sample_i = np.random.choice(range(X.shape[0]), shadow_dataset_size, replace=False)
    assert np.unique(sample_i).shape[0] == shadow_dataset_size # sanity check
    D_shadows.append((X[sample_i, :], y[sample_i, :]))
  return D_shadows

# returns a list of 'n_shadows' datasets
def generate_shadow_dataset(target_model, n_shadows, shadow_dataset_size, n_classes, X_test=None, y_test=None):
  # in case we give test data we will just divide those to train the shadow models
  if X_test is not None and y_test is not None:
    return divide_dataset(n_shadows, shadow_dataset_size, X_test, y_test)
  
  
  # helper function to return a datapoint (for sure)
  def get_shadow_datapoint(c):
    X_i = None
    with tf.device('/gpu:0'):
      while X_i is None:
        X_i = synthesize(c, target_model, 1, 32*32*3, 0.65, 100, 5)
    return X_i
  
  D_shadows = []
  for i in range(n_shadows):
    print(f"Generating D_shadow_{i}")
    # uniformly generate X's for all of the classes (y's)
    X_shadow = np.asarray(
        [get_shadow_datapoint(i%n_classes) for i in range(shadow_dataset_size)]
    )
    y_shadow = np.asarray(
        [(i%n_classes) for i in range(shadow_dataset_size)]
    ).reshape((-1, 1))

    D_shadows.append((X_shadow, y_shadow))
  
  return D_shadows

def create_shadows(D_shadows):
  shadow_models = [] # shadow model list

  for D_shadow in D_shadows:
    # sample data to feed/evaluate the model
    X_shadow, y_shadow = D_shadow
    shadow_X_train, shadow_X_test, shadow_y_train, shadow_y_test = train_test_split(X_shadow, y_shadow, shuffle=True, test_size=0.33)

    # generate the shadow model
    shadow_model = f_shadow(shadow_X_train, shadow_y_train, shadow_X_test, shadow_y_test)

    D_shadow = (shadow_X_train, shadow_y_train), (shadow_X_test, shadow_y_test)
    shadow_models.append((shadow_model, D_shadow))

  return shadow_models # return a list where every item is (model, acc), train-data, test-data

In [42]:
# generate shadow datasets
D_shadows = generate_shadow_dataset(target_model, 20, 2500, 10, test_images, test_labels)

In [43]:
# train the shadow models
shadow_models = create_shadows(D_shadows)

Epoch 1/100
53/53 [==============================] - 2s 17ms/step - loss: 5.3825 - accuracy: 0.1045 - val_loss: 2.2975 - val_accuracy: 0.1345
Epoch 2/100
53/53 [==============================] - 1s 13ms/step - loss: 2.2911 - accuracy: 0.1301 - val_loss: 2.2944 - val_accuracy: 0.1248
Epoch 3/100
53/53 [==============================] - 1s 12ms/step - loss: 2.2873 - accuracy: 0.1469 - val_loss: 2.2749 - val_accuracy: 0.1588
Epoch 4/100
53/53 [==============================] - 1s 12ms/step - loss: 2.2381 - accuracy: 0.1600 - val_loss: 2.2291 - val_accuracy: 0.1758
Epoch 5/100
53/53 [==============================] - 1s 13ms/step - loss: 2.2010 - accuracy: 0.1719 - val_loss: 2.1705 - val_accuracy: 0.2036
Epoch 6/100
53/53 [==============================] - 1s 12ms/step - loss: 2.1296 - accuracy: 0.2209 - val_loss: 2.0593 - val_accuracy: 0.2242
Epoch 7/100
53/53 [==============================] - 1s 13ms/step - loss: 2.0044 - accuracy: 0.2460 - val_loss: 1.9379 - val_accuracy: 0.2497
Epoch 

In [44]:
def __f_attack(X_train, y_train, X_test, y_test, epochs=50):
  print(X_train.shape, X_test.shape)
  model = models.Sequential()
  model.add(layers.Dense(11, activation='relu', input_shape=(X_train.shape[1], )))
  model.add(layers.Dense(100, activation='relu'))
  model.add(layers.Dense(1000, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  history = model.fit(X_train, y_train, epochs=epochs, 
                    validation_data=(X_test, y_test), verbose=True)
  
  return model

def f_attack(X, y):
  # X_i = (class, probability vector, )
  classes = np.unique(train_labels) # all class labels
  with tf.device('/gpu:0'):
  # split to train and test datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2)
    attack_model = __f_attack(X_train, y_train, X_test, y_test)

  return attack_model

In [45]:
# helper function to prepare each shadow dataset batch
def prepare_batch(shadow_model, X, y, in_D):
  #decide membership
  y_member = np.ones(shape=(y.shape[0], 1)) if in_D else np.zeros(shape=(y.shape[0], 1))
  
  # get probability vector
  prob_layer = layers.Softmax() # probability layer implementing softmax for mapping NN results to probabilities in [0, 1]
  prob_vec = prob_layer(shadow_model(X)).numpy()
  
  # return an instance <actual class, prob_vec from shadow model, 'in'/'out' D_target membership> 
  return np.concatenate((y.reshape(-1, 1), prob_vec, y_member), axis=1)

def generate_attack_dataset(shadow_models, n_classes):
  # input is a list where items are model, (X_train, y_train), (X_test, y_test)

  D_attack = None
  # D_attack_i format = <class, prob_vec, membership label (1 or 0)> 
  for shadow_model, ((X_train, y_train), (X_test, y_test)) in shadow_models:
    s = min(X_train.shape[0], X_test.shape[0])
    batch = np.concatenate((
        prepare_batch(shadow_model, X_train[:s], y_train[:s], True), # members of shadow dataset 
        prepare_batch(shadow_model, X_test[:s], y_test[:s], False)   # non members of shadow dataset
    ))   

    D_attack = np.concatenate((D_attack, batch)) if D_attack is not None else batch  

  return D_attack 

In [46]:
D_attack = generate_attack_dataset(shadow_models, 10)

In [47]:
attack_model_bundle = f_attack(D_attack[:, :-1], D_attack[:, -1])

(26400, 11) (6600, 11)
Epoch 1/50
825/825 [==============================] - 4s 4ms/step - loss: 0.5372 - accuracy: 0.7317 - val_loss: 0.4768 - val_accuracy: 0.7864
Epoch 2/50
825/825 [==============================] - 3s 4ms/step - loss: 0.4799 - accuracy: 0.7791 - val_loss: 0.4610 - val_accuracy: 0.7914
Epoch 3/50
825/825 [==============================] - 3s 4ms/step - loss: 0.4692 - accuracy: 0.7834 - val_loss: 0.4639 - val_accuracy: 0.7929
Epoch 4/50
825/825 [==============================] - 3s 4ms/step - loss: 0.4644 - accuracy: 0.7858 - val_loss: 0.4521 - val_accuracy: 0.8002
Epoch 5/50
825/825 [==============================] - 3s 4ms/step - loss: 0.4588 - accuracy: 0.7890 - val_loss: 0.4464 - val_accuracy: 0.7994
Epoch 6/50
825/825 [==============================] - 3s 4ms/step - loss: 0.4549 - accuracy: 0.7911 - val_loss: 0.4469 - val_accuracy: 0.7982
Epoch 7/50
825/825 [==============================] - 3s 4ms/step - loss: 0.4538 - accuracy: 0.7898 - val_loss: 0.4400 - val_

In [48]:
def evaluate_attack(attack_model, X_attack, y_attack, n_classes):
  acc_per_class = []
  for c in range(n_classes):
    class_instances = X_attack[:, 0] == c # get same class samples
    test_loss, test_acc = attack_model.evaluate(X_attack[class_instances, :], y_attack[class_instances], verbose=0)
    acc_per_class.append(test_acc)
    print(f"class-{c+1}: {test_acc}")
  return acc_per_class



In [49]:
# create a test dataset 

D_out = prepare_batch(target_model, test_images[:2500], test_labels[:2500], False)
D_in = prepare_batch(target_model, train_images, train_labels, True)
print("Testing with 'in' data only:")
res_in = evaluate_attack(attack_model_bundle, D_in[:, :-1], D_in[:, -1], 10)

print("\nTesting with 'out' data only:")
res_out = evaluate_attack(attack_model_bundle, D_out[:2500, :-1], D_out[:2500, -1], 10)

print("\nTesting with all prev data: ")
res_all = evaluate_attack(attack_model_bundle, np.concatenate((D_out[:2500, :-1], D_in[:, :-1])), np.concatenate((D_out[:2500, -1], D_in[:, -1])), 10)

print(f"\nTotal attack accuracy: {np.mean(res_all)}")

Testing with 'in' data only:
class-1: 0.8695651888847351
class-2: 0.8641975522041321
class-3: 0.8392156958580017
class-4: 0.8524590134620667
class-5: 0.8816794157028198
class-6: 0.8262711763381958
class-7: 0.8840000033378601
class-8: 0.8616600632667542
class-9: 0.874015748500824
class-10: 0.9039999842643738

Testing with 'out' data only:
class-1: 0.670634925365448
class-2: 0.6543778777122498
class-3: 0.7196969985961914
class-4: 0.8016529083251953
class-5: 0.731517493724823
class-6: 0.726190447807312
class-7: 0.643122673034668
class-8: 0.6266094446182251
class-9: 0.6015037298202515
class-10: 0.5927419066429138

Testing with all prev data: 
class-1: 0.7702970504760742
class-2: 0.7652173638343811
class-3: 0.7784200310707092
class-4: 0.8271604776382446
class-5: 0.8073217868804932
class-6: 0.7745901346206665
class-7: 0.7591522336006165
class-8: 0.748971164226532
class-9: 0.7346153855323792
class-10: 0.7489959597587585

Total attack accuracy: 0.7714741587638855
